## 1.데이터 로드

In [1]:
import pandas as pd

data = pd.read_csv("/home/lab543/jupyter/jinyoung/data/abcnews-date-text.csv",error_bad_lines=False)

len(data)

1186018

In [2]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [3]:
text = data[['headline_text']]
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


## 2.전처리

### 1)토큰화

In [4]:
import nltk

text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

/home/lab543/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


NLTK의 word_tokenize를 통해 단어 토큰화 수행

In [5]:
text.head(5)

,headline_text
0,"[aba, decides, against, community, broadcastin..."
1,"[act, fire, witnesses, must, be, aware, of, de..."
2,"[a, g, calls, for, infrastructure, protection,..."
3,"[air, nz, staff, in, aust, strike, for, pay, r..."
4,"[air, nz, strike, to, affect, australian, trav..."


### 2)불용어 처리

In [6]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])

/home/lab543/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
text.head(5)

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


### 3)표제어 추출(3인칭 -> 1인칭, 과거형 -> 현재형)

In [8]:
from nltk.stem import WordNetLemmatizer

text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head(5)

/home/lab543/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,headline_text
0,"[aba, decide, community, broadcast, licence]"
1,"[act, fire, witness, must, aware, defamation]"
2,"[g, call, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


### 4) 길이 3이하 제거

In [9]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
tokenized_doc[:5]

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object

### 5) TF-IDF 통계량 사용

In [10]:
# 역토큰화 작업
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
text['headline_text'] = detokenized_doc

/home/lab543/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [11]:
text.head(5)

,headline_text
0,decide community broadcast licence
1,fire witness must aware defamation
2,call infrastructure protection summit
3,staff aust strike rise
4,strike affect australian travellers


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000) # 상위 1,000개의 단어를 보존 

X = vectorizer.fit_transform(text['headline_text'])
X.shape

(1186018, 1000)

1,082,168 × 1,000의 크기를 가진 가진 TF-IDF 행렬이 생겼습니다. 이제 이에 LDA를 수행합니다.

## 3. 토픽 모델링

In [17]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X) #10개 토픽으로 나눈다.

In [18]:
print(lda_model.components_)
print(lda_model.components_.shape) 

[[1.00001251e-01 1.00000870e-01 1.00000959e-01 ... 1.00003789e-01
  1.00005244e-01 1.00005701e-01]
 [1.00001186e-01 1.00000321e-01 1.00001492e-01 ... 1.00008495e-01
  1.00003773e-01 5.28131364e+02]
 [1.00002566e-01 1.00000691e-01 1.00001989e-01 ... 1.00004725e-01
  1.00004888e-01 1.00003502e-01]
 ...
 [1.00001872e-01 1.00000609e-01 1.00004551e-01 ... 1.00006062e-01
  1.00004418e-01 1.00005045e-01]
 [1.39239876e+02 1.00000829e-01 1.00002009e-01 ... 1.00005526e-01
  1.00004846e-01 1.00004377e-01]
 [1.00001172e-01 2.98823905e+02 1.00002656e-01 ... 1.00006216e-01
  1.00003730e-01 1.00006511e-01]]
(10, 1000)


In [19]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('sydney', 9617.34), ('queensland', 8104.47), ('kill', 7338.07), ('court', 6475.52), ('open', 5572.5)]
Topic 2: [('australia', 15449.55), ('australian', 13312.79), ('government', 7949.7), ('home', 6582.47), ('leave', 4909.81)]
Topic 3: [('donald', 7654.72), ('live', 6199.76), ('south', 5924.7), ('federal', 4876.11), ('help', 4841.53)]
Topic 4: [('melbourne', 7084.73), ('canberra', 6100.01), ('report', 5558.55), ('people', 5271.16), ('time', 4731.3)]
Topic 5: [('police', 13274.17), ('attack', 6849.04), ('speak', 5367.5), ('family', 5250.09), ('warn', 5147.3)]
Topic 6: [('house', 6402.18), ('test', 5756.46), ('tasmania', 5385.18), ('plan', 4782.06), ('talk', 4215.42)]
Topic 7: [('charge', 8704.62), ('murder', 6698.9), ('shoot', 6287.13), ('years', 6079.1), ('north', 5422.0)]
Topic 8: [('trump', 15036.78), ('death', 6809.39), ('change', 6625.03), ('crash', 6418.79), ('year', 6139.47)]
Topic 9: [('election', 8921.67), ('market', 7054.2), ('make', 6682.57), ('adelaide', 6153.72), 